**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2024*

----

# Combinación de conjuntos de datos

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [2]:
import plotly
plotly.__version__


'5.19.0'

In [ ]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [3]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
melb_df.shape

(13580, 21)

## Datos adicionales

Vamos a aumentar los datos presentes en el conjunto dado con un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor precisión el valor del vecindario de cada propiedad.

Para ello, utilizaremos [un conjunto de datos](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

In [5]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

C:\Users\emanu\AppData\Local\Temp\ipykernel_19024\172457013.py:9: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pandas.read_csv(


In [6]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


In [7]:
len(airbnb_df)

22895

Para poder realizar esta combinación de datos, tenemos que encontrar una columna que contenga información en común. Utilizaremos el zipcode.

* ¿Qué desventajas tiene utilizar el zipcode para agregar información sobre la vecindad de cada propiedad en venta? ¿Se corresponde con lo que entendemos intuitivamente como vecindad?
* ¿Qué otros métodos podrían utilizarse?

In [8]:
airbnb_df.zipcode.value_counts()[:5]

zipcode
3000.0    2491
3000       876
3006.0     860
3182.0     579
3182       556
Name: count, dtype: int64

 Sin embargo, este zipcode tiene tipos diferentes, que tendremos que estandarizar antes de continuar con la combinación de los datos.

In [9]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

In [10]:
airbnb_df.zipcode.value_counts().head(5)

zipcode
3000.0    3367
3006.0    1268
3182.0    1135
3141.0     881
3004.0     728
Name: count, dtype: int64

In [11]:
airbnb_df.zipcode.dtype

dtype('float64')

In [12]:
airbnb_df['zipcode_int'] = airbnb_df.zipcode.fillna(0).astype('int')

In [13]:
airbnb_df[airbnb_df['zipcode_int'] == 0].shape

(146, 14)

### Valores en común

Antes de unir los conjuntos de datos, tenemos que asegurarnos de que tengamos suficientes registros en común como para agregar información relevantes.

In [14]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

Airbnb unique zipcodes 248
Sales unique zipcodes 198
Common zipcodes 191


In [15]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.9985272459499264
Records in Airbnb df with corresponding zipcode form Sales df 0.9302904564315353


### Exploración visual

Para terminar de asegurarnos de que las áreas representadas por ambos datasets son consitentes, podemos representar las coordenadas en un mapa usando Plotly.

Aunque estas visualizaciones no son completas en cuanto a la información geográfica que muestran, aproximadamente podemos ver que se tratan de zonas similares.

In [17]:
import plotly.express as px
data = melb_df.sample(300)
fig = px.scatter_geo(
    data, lat=data.Lattitude, lon=data.Longtitude, color=data.YearBuilt
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [18]:
import plotly.express as px
color_col = 'state'
data = airbnb_df[~airbnb_df[color_col].isna()].sample(300)
fig = px.scatter_geo(
    data, lat=data.latitude, lon=data.longitude, color=data[color_col]
    )
fig.update_geos(fitbounds="locations")
fig.show()

## Operación merge

Es hora de combinar ambos conjuntos de datos. Una opción frecuentemente tentadora es aplicar la operación merge sin analizar demasiado la situación. Sin embargo, ¡esta operación puede ocasionar una **explosión exponencial** de nuestro conjunto de datos!

¿Por qué la siguiente operación genera un DataFrame con 2 millones de filas?

In [19]:
## Incorrecto!
len(melb_df.merge(airbnb_df, how='left',
                  left_on='Postcode', right_on='zipcode'))

2139684

In [20]:
incorrect_df = melb_df.merge(airbnb_df, how='left',
                  left_on='Postcode', right_on='zipcode')

In [21]:
melb_df[melb_df['Address'] == "85 Turner St"]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0


In [22]:
airbnb_df[airbnb_df['zipcode'] == 3067].shape

(258, 14)

In [23]:
incorrect_df[incorrect_df['Address'] == "85 Turner St"]['description']

0      This is a space to enjoy, a place to come and ...
1      Melbourne Inner city living at its very best!!...
2      Set in a relaxing, bright city garden, complet...
3      Cosy furnished single bedroom in a retro flat....
4      My place is close to the Yarra River and the f...
                             ...                        
253    Contemporary, well-equipped, comfortable townh...
254    Enjoy simple luxury living in my newly furnish...
255    The place is located near Richmond and is a 10...
256    Experience modern riverside living in my light...
257    Charming, bright, clean and comfortable house ...
Name: description, Length: 258, dtype: object

Recordemos que nuestro objetivo es agregar información sobre el valor del vecindario de la propiedad. ¿Qué información del conjunto de airbnb nos sirve para representar esto?

* Promedio de `price` por zipcode
* Promedio de `weekly_price` por zipcode
* Promedio de `monthly_price` por zipcode

Queda a interpretación de ustedes decidir cuál de todos ellos utilizar.

In [24]:
relevant_cols = ['price', 'weekly_price', 'monthly_price']

Cuando queremos agregar un único dato luego de agrupar un DataFrame, la operación es bastante sencilla. Sólo tienen que tener en cuenta que el nombre de la columnas será el mismo, aunque su valor ahora esté agregado.

In [25]:
# Option 1: inefficient form.
# The aggregation is calculated over all columns, and after is calculated
# we filter only the columns we want.
# airbnb_df.groupby('zipcode').mean()[relevant_cols].reset_index()[:5]

# Option 2: efficient form
# First we filter out the columns we want, and then we only aggregate
# those. Be careful to include the grouping column as well.
airbnb_df[relevant_cols + ['zipcode']]\
  .groupby('zipcode').mean().reset_index()[:5]

,zipcode,price,weekly_price,monthly_price
0,2010.0,40.000000,NaN,NaN
1,2134.0,50.000000,NaN,NaN
2,2582.0,104.000000,NaN,NaN
3,3000.0,150.504307,918.738956,3407.204651
4,3001.0,132.500000,NaN,NaN


Agrupar y agregar utilizando más de un tipo de operación es un poco más complicado. Pandas nos provee de muchas formas de escribir la misma operación. Ninguna de ellas es muy intuitiva, y usualmente requiere renombrar las columnas para darles nombres más significativos.

In [26]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()

In [ ]:
airbnb_price_by_zipcode

zipcode       price       weekly_price monthly_price
                    mean count         mean          mean
0     2010.0   40.000000     1          NaN           NaN
1     2134.0   50.000000     1          NaN           NaN
2     2582.0  104.000000     1          NaN           NaN
3     3000.0  150.504307  3367   918.738956   3407.204651
4     3001.0  132.500000     2          NaN           NaN
..       ...         ...   ...          ...           ...
242   7367.0  230.000000     1          NaN           NaN
243   8938.0  296.000000     1          NaN           NaN
244   9004.0  226.000000     1          NaN           NaN
245   9540.0  245.000000     1          NaN           NaN
246  30122.0   75.000000     1   525.000000   2281.000000

[247 rows x 5 columns]

In [27]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

## Option 2
# Pass as argument name the new name of the column, and as value a tuple where
# the first value is the original column and the second value is the operation.
# airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
#   .agg(airbnb_record_count=('price', 'count'),
#        airbnb_price_mean=('price', 'mean'),
#        airbnb_weekly_price_mean=('weekly_price', 'mean'),
#        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
#   .reset_index()

airbnb_price_by_zipcode[:3]

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.0,1,NaN,NaN
1,2134.0,50.0,1,NaN,NaN
2,2582.0,104.0,1,NaN,NaN


Guardamos el dataset construido ya que nos sera util para el entregable

In [28]:
airbnb_price_by_zipcode.to_csv("airbnb_price_by_zipcode.csv", index=None)

In [29]:
# Para Google Colab

# from google.colab import files
# files.download('airbnb_price_by_zipcode.csv')

Finalmente hacemos la combinacion de ambos datasets creados

In [30]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
992,Box Hill,12 Pendle St,4,h,1506000.0,S,hockingstuart,17/09/2016,13.1,3128.0,...,Whitehorse,-37.81800,145.13540,Eastern Metropolitan,4605.0,3128.0,111.443925,214.0,426.375000,1610.625000
7845,Fitzroy North,58 Scotchmer St,2,h,1345000.0,S,Woodards,8/04/2017,3.5,3068.0,...,Yarra,-37.78310,144.98050,Northern Metropolitan,6244.0,3068.0,126.012987,308.0,801.315789,2801.842105
10628,Collingwood,30 Alexander St,3,h,1297000.0,S,Dixon,8/07/2017,2.0,3066.0,...,Yarra,-37.79557,144.99158,Northern Metropolitan,4553.0,3066.0,145.989899,297.0,920.078947,3087.230769
3269,Heidelberg Heights,3/47 Southern Rd,2,h,510000.0,SP,Fletchers,27/11/2016,10.5,3081.0,...,Banyule,-37.74330,145.05450,Eastern Metropolitan,2947.0,3081.0,83.657895,38.0,291.428571,937.500000
10659,Epping,27 Hammond Dr,3,h,490000.0,SP,Ristic,8/07/2017,19.6,3076.0,...,Whittlesea,-37.64166,145.02885,Northern Metropolitan,10926.0,3076.0,51.529412,17.0,395.000000,1440.000000


In [31]:
merged_sales_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'zipcode',
       'airbnb_price_mean', 'airbnb_record_count', 'airbnb_weekly_price_mean',
       'airbnb_monthly_price_mean'],
      dtype='object')

In [32]:
merged_sales_df.to_csv("melb_data_extended.csv", index=None)

In [33]:
# Para Google Colab

# from google.colab import files
# files.download('melb_data_extended.csv')